- Chargement du fichier qui contient la description de la pose "keypoints"
- Créer un tenseur topology pour définir comment les keypoints sont liés entre eux.

In [ ]:
import json
import trt_pose.coco
import math

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

- Définir la résolution de l'entrée 224x224
- Initialisé l'entrée avec un example qui contient que des zeros

In [ ]:
import torch
WIDTH = 224
HEIGHT = 224
data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

- charger les poids du réseau optimisé
Remarque: ce réseau a été optimisé en utilisant torch2trt

In [ ]:
from torch2trt import TRTModule
OPTIMIZED_MODEL = 'mon_model.pth'
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

 - Importer les bibliothéques pour le traitement d'images (cv2), ainsi pour le prétraitement (torchvision.transforms,PIL.image)
 - Définir les tenseurs de la moyenne, écart-type pour la normalisation de l'image et créer une allocation pour ces tenseur dans le GPU (torch.device('cuda'))
 la fonction preprocess:
     - Convertit l'image en RGB
     - Convertir l'image en PIL image dans le but de le transformer après en tenseur (voir ligne d'après)
     - Normaliser l'image

In [ ]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

- "parse_objects" classe pour renvoyer les keypoints trouvés à partir du réseau de neurones
- "draw_objects" classe pour dessiner ces points

In [ ]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

- utiliser la biblithéque jetcam (sur nvidia seulement) pour récupérer une image BGR 

In [ ]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=WIDTH, height=HEIGHT,capture_fps=2)

camera.running = True

- Initialiser 4 tenseurs pytorch pour enregistrer les coordonnées des keypoints récupérés

In [ ]:
# intialisation des tenseurs pytorch pour le stockage 
x,y,x1,y1 = torch.zeros([1,18]),torch.zeros([1,18]),torch.zeros([1,18]),torch.zeros([1,18])

#initialisation du tenseur pour le comptage
cpt = torch.zeros([1,4])
    
def stocker(tenseur_x,tenseur_y):
    global x1,y1
    x1 = torch.cat((x1,tenseur_x),0)
    y1 = torch.cat((y1,tenseur_y),0)
    return x1,y1



In [ ]:
# Importer les classes
from Mouvements import Mouvements
from Manipulation import Manipulation

# Création des objets instances
mvt = Mouvements()
manip = Manipulation()


Fonction pour récupérer les keypoints 

In [ ]:
def get_keypoints(image, human_pose, topology, object_counts, objects, normalized_peaks):
    global x,y

    height = image.shape[0]
    width = image.shape[1]
    count = int(object_counts[0])# il y a combien de personnes détéctés
    
    
    for i in range(count):# parcourir les personnes détéctées
        obj = objects[0][i] # obj determine les points clés détéctés
        # obj est -1 si le keypoint n'existe pas et 0 s'il existe
        C = obj.shape[0] # le nombre des keypoints (toujours C= 18)
        for j in range(C): #parcourir les 18 points clés j est le nombre de l'indice du keypoint
                k = int(obj[j]) 
                if k >= 0: #si le keypoint existe
                    peak = normalized_peaks[0][j][k] #on associe le keypoint qui existe à l'indice du keypoint J
                    x[0][j] = round(float(peak[1]) * width)
                    y[0][j] = round(float(peak[0]) * height)
                else:
                    #quand on a pas une detection on rempli par la derniere valeur qu'on a obtenu
                    x[0][j] = x1[-1][j]
                    y[0][j] = y1[-1][j]
                    
    # Stocker les Coordonnées                
    stocker(x,y) 
    # Renvoyer l'angle du bras gauche
    angle_gauche = manip.getAngle(x1,y1,9,7,5)
    # Détécter l'uppercut gauche
    uppercut_gauche = mvt.uppercut(y1,angle_gauche,'gauche',cpt)
    # Renvoyer l'angle du bras droit
    angle_droite = manip.getAngle(x1,y1,10,8,6)
    # Détécter l'uppercut droite
    uppercut_droite = mvt.uppercut(y1,angle_droite,'droite',cpt)
    #Detecter le JAB
    JAB = mvt.JAB(angle_gauche,y1,'gauche',cpt)
    #Detecter le contre
    Cross = mvt.JAB(angle_droite,y1,'droite',cpt)
    #rajouter les coups dans le tenseur combinaison (comb)
    manip.combinaisons(JAB,Cross,uppercut_gauche,uppercut_droite)
    
    


Fonction d'éxécution pour charger le nouvelle l'image,effectuer un pretraitement,detecter les keypoints, convertir l'image en JPEG et enfin appeler la fonction get_keypoints

In [ ]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    keypoints = get_keypoints(image, human_pose, topology, counts, objects,peaks)

- une interface pour la visualisation en temps réel

In [ ]:
import ipywidgets
from IPython.display import display
image_w = ipywidgets.Image(format='jpeg',width = 500,height = 300)
display(image_w)

Lancer la caméra

In [ ]:
camera.observe(execute, names='value')

Arréter la caméra

In [ ]:
camera.unobserve_all()
camera.running = False
camera.cap.release()

#bilan des mouvements
manip.statistiques(cpt)
manip.send(cpt)

In [ ]:
print("""Le nombre total des mouvements effectués est {} ({} JAB , {} CROSS , {} Uppercuts gauches ,{} Uppercuts Droites) \nPourcentages :\n JAB : {:.1f}%\n CROSS : {:.1f}%\n Uppercuts Gauches : {:.1f}% \n Uppercuts Droites : {:.1f}%\n Les Enchainements détéctés :\n JAB JAB CROSS : {}\n JAB CROSS Uppercut_gauche : {}\n JAB CROSS JAB :{}\nJAB CROSS Uppercut_droite {}""".format(int(Manipulation.mvt_pourcentage[0]),int(cpt[0,2]),int(cpt[0,3]),int(cpt[0,0]),int(cpt[0,1]),Manipulation.mvt_pourcentage[1],Manipulation.mvt_pourcentage[2],Manipulation.mvt_pourcentage[3],Manipulation.mvt_pourcentage[4],int(Manipulation.comb[0,0]),int(Manipulation.comb[0,1]),int(Manipulation.comb[0,2]),int(Manipulation.comb[0,3])))